# Bước 6: Phân cụm khách hàng từ Luật kết hợp (Association Rules → Clustering)

Notebook này lấy **kết quả luật kết hợp** (Apriori/FP-Growth) và biến chúng thành **đặc trưng** để phân cụm khách hàng bằng K-Means.

## Ý tưởng cốt lõi
- Mỗi luật có dạng: **Antecedent → Consequent**
- Với mỗi khách hàng, ta kiểm tra: khách đó đã từng mua **đủ antecedents** của luật hay chưa.
- Mỗi luật trở thành một feature (0/1 hoặc có trọng số theo lift/confidence).
- (Tuỳ chọn) Ghép thêm **RFM** để phân cụm ổn định hơn.


## Parameters
Gán tham số để chạy bằng papermill.


In [ ]:
# PARAMETERS (for papermill)

# Input
CLEANED_DATA_PATH = "data/processed/cleaned_uk_data.csv"
RULES_INPUT_PATH = "data/processed/rules_apriori_filtered.csv"  # hoặc rules_fpgrowth_filtered.csv

# Feature engineering
TOP_K_RULES = 200
SORT_RULES_BY = "lift"      # lift | confidence | support
WEIGHTING = "lift"          # none | lift | confidence | support | lift_x_conf
MIN_ANTECEDENT_LEN = 1
USE_RFM = True
RFM_SCALE = True
RULE_SCALE = False

# Clustering
K_MIN = 2
K_MAX = 10
N_CLUSTERS = None            # None => chọn theo silhouette, hoặc đặt số cụ thể (vd 5)
RANDOM_STATE = 42

# Output
OUTPUT_CLUSTER_PATH = "data/processed/customer_clusters_from_rules.csv"

# Visual
PROJECTION_METHOD = "pca"   # pca | svd
PLOT_2D = True


## Set up


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CẤU HÌNH GIAO DIỆN ĐẸP (Copy đoạn này vào đầu notebook)
sns.set_theme(style="whitegrid")  # Nền trắng kẻ lưới
plt.rcParams['figure.figsize'] = (12, 6) # Biểu đồ to rộng
plt.rcParams['figure.dpi'] = 100         # Độ nét cao
plt.rcParams['font.size'] = 12           # Chữ to dễ đọc
# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from cluster_library import RuleBasedCustomerClusterer


## Load cleaned data & rules


In [ ]:
df_clean = pd.read_csv(CLEANED_DATA_PATH, parse_dates=["InvoiceDate"])
print(df_clean.shape)
df_clean.head()


In [ ]:
clusterer = RuleBasedCustomerClusterer(df_clean=df_clean)
customer_item_bool = clusterer.build_customer_item_matrix(threshold=1)
print('Customer × Item:', customer_item_bool.shape)

rules_df = clusterer.load_rules(
    rules_csv_path=RULES_INPUT_PATH,
    top_k=TOP_K_RULES,
    sort_by=SORT_RULES_BY,
)
print('Rules used:', rules_df.shape)
rules_df.head()


## Build features (Rules → Features) + (optional) RFM


In [ ]:
X, meta = clusterer.build_final_features(
    weighting=WEIGHTING,
    use_rfm=USE_RFM,
    rfm_scale=RFM_SCALE,
    rule_scale=RULE_SCALE,
    min_antecedent_len=MIN_ANTECEDENT_LEN,
)
print('X shape:', X.shape)
meta.head()


## Choose K (silhouette)


In [ ]:
sil_df = clusterer.choose_k_by_silhouette(
    X,
    k_min=K_MIN,
    k_max=K_MAX,
    random_state=RANDOM_STATE,
)
sil_df


In [ ]:
best_k = int(sil_df.loc[0, 'k'])
k = best_k if N_CLUSTERS is None else int(N_CLUSTERS)
print('Chosen k =', k)


## Fit KMeans & save results


In [ ]:
labels = clusterer.fit_kmeans(X, n_clusters=k, random_state=RANDOM_STATE)
meta_out = meta.copy()
meta_out['cluster'] = labels

# Lưu
os.makedirs(os.path.dirname(OUTPUT_CLUSTER_PATH), exist_ok=True)
meta_out.to_csv(OUTPUT_CLUSTER_PATH, index=False)
print('Saved:', OUTPUT_CLUSTER_PATH)
meta_out.head()


## Quick profiling


In [ ]:
profile_cols = ['cluster'] + ([c for c in ['Recency','Frequency','Monetary'] if c in meta_out.columns])
summary = meta_out.groupby('cluster').agg({
    'CustomerID': 'count',
    **{c:'mean' for c in profile_cols if c!='cluster'}
}).rename(columns={'CustomerID':'n_customers'}).sort_values('n_customers', ascending=False)
summary


## 2D visualization (PCA/SVD)


In [ ]:
if PLOT_2D:
    Z = clusterer.project_2d(X, method=PROJECTION_METHOD, random_state=RANDOM_STATE)
    plt.figure(figsize=(8,6))
    plt.scatter(Z[:,0], Z[:,1], c=labels, s=10)
    plt.title('Customer clusters (2D projection)')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.tight_layout()
    plt.show()


In [ ]:
# --- ĐOẠN CODE NÂNG CẤP: SO SÁNH THUẬT TOÁN (Đã sửa lỗi import) ---
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import pandas as pd
import numpy as np

print("\n=== BẮT ĐẦU SO SÁNH CÁC THUẬT TOÁN (Yêu cầu nâng cao 2.3) ===")

# 1. Lấy dữ liệu X đã chuẩn hóa
# Nếu X chưa được định nghĩa ở cell này, ta lấy từ biến toàn cục hoặc thuộc tính của clusterer nếu có
# Ở đây giả định biến X (features) vẫn còn lưu trong bộ nhớ từ các cell trên
try:
    X_final = X.values if hasattr(X, 'values') else X
except NameError:
    print("Lỗi: Không tìm thấy biến dữ liệu X. Hãy đảm bảo bạn đã chạy các cell bên trên!")
    # Fallback nếu cần thiết (thường không cần nếu chạy tuần tự)
    X_final = None

if X_final is not None:
    # 2. Định nghĩa các model để so sánh
    k_best = 3 # Giả sử K=3 (hoặc số K bạn thấy tốt nhất)

    models = {
        "K-Means (Baseline)": KMeans(n_clusters=k_best, random_state=42, n_init='auto'),
        "Agglomerative": AgglomerativeClustering(n_clusters=k_best),
        "DBSCAN": DBSCAN(eps=0.5, min_samples=5) 
    }

    results = []
    print(f"Đang chạy so sánh trên {X_final.shape[0]} khách hàng...")

    for name, model in models.items():
        try:
            # Fit model
            labels = model.fit_predict(X_final)
            
            # Xử lý nhiễu của DBSCAN (nhãn -1)
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            
            if n_clusters < 2:
                print(f"  - {name}: Chỉ tìm thấy {n_clusters} cụm (hoặc toàn nhiễu) -> Bỏ qua.")
                continue
                
            # Tính toán Metrics
            sil = silhouette_score(X_final, labels)
            ch = calinski_harabasz_score(X_final, labels)
            
            results.append({
                "Thuật toán": name,
                "Số cụm": n_clusters,
                "Silhouette Score": round(sil, 4),
                "Calinski-Harabasz": round(ch, 2)
            })
            print(f"  - {name}: Xong (Sil={sil:.3f})")
            
        except Exception as e:
            print(f"  - {name}: Lỗi - {str(e)}")

    # 3. Hiển thị bảng kết quả
    if results:
        comparison_df = pd.DataFrame(results)
        print("\n>>> BẢNG TỔNG HỢP SO SÁNH HIỆU NĂNG:")
        display(comparison_df)
    else:
        print("Không có kết quả so sánh nào hợp lệ.")

In [ ]:
# --- CODE YÊU CẦU 5: SO SÁNH CÁC CẤU HÌNH ĐẶC TRƯNG (ĐÃ FIX LỖI) ---
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.cluster import KMeans
import pandas as pd

print("\n=== BẮT ĐẦU CHẠY SO SÁNH (YÊU CẦU 5) ===")

# Định nghĩa 3 kịch bản
scenarios = [
    {"name": "1. Rule-Only (Binary)", "weighting": None, "use_rfm": False},
    {"name": "2. Rule-Only (Weighted Lift)", "weighting": "lift", "use_rfm": False},
    {"name": "3. Rules + RFM (Hybrid)", "weighting": "lift", "use_rfm": True}
]

results = []

try:
    for scen in scenarios:
        print(f"Đang chạy kịch bản: {scen['name']}...")
        
        # 1. Tạo lại ma trận đặc trưng (X) theo cấu hình
        # SỬA LỖI: Gọi giống hệt Ô 5, không truyền rules_df/rfm_df từ ngoài vào
        X_temp, _ = clusterer.build_final_features(
            weighting=scen["weighting"],
            use_rfm=scen["use_rfm"],
            # Các tham số mặc định lấy từ biến toàn cục (Ô 1)
            rfm_scale=RFM_SCALE,
            rule_scale=RULE_SCALE,
            min_antecedent_len=MIN_ANTECEDENT_LEN,
        )
        
        # 2. Chạy K-Means (Fix K=3 để so sánh công bằng)
        # Lưu ý: k_best lấy từ kết quả Ô 7
        k_eval = k if 'k' in globals() else 3 
        model = KMeans(n_clusters=k_eval, random_state=42, n_init='auto')
        labels = model.fit_predict(X_temp)
        
        # 3. Tính điểm
        sil = silhouette_score(X_temp, labels)
        ch = calinski_harabasz_score(X_temp, labels)
        
        results.append({
            "Cấu hình": scen["name"],
            "Silhouette Score": round(sil, 4),
            "Calinski-Harabasz": round(ch, 2)
        })

    # In bảng kết quả
    print("\n>>> BẢNG KẾT QUẢ SO SÁNH:")
    display(pd.DataFrame(results))

except Exception as e:
    print(f"Lỗi: {e}")

In [ ]:
# --- CODE YÊU CẦU 6: PHÂN TÍCH CHI TIẾT CỤM (PROFILING) ---

# 1. Load lại file kết quả vừa chạy (đảm bảo lấy file mới nhất)
df_res = pd.read_csv("data/processed/customer_clusters_from_rules.csv")

print("\n=== THỐNG KÊ RFM THEO CỤM ===")
rfm_stats = df_res.groupby('cluster').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).rename(columns={'CustomerID': 'Số Lượng Khách'}).round(1)
display(rfm_stats)

# 2. Tìm "Dấu hiệu đặc trưng" (Luật nào kích hoạt nhiều nhất ở mỗi cụm?)
# (Phần này mô phỏng logic tìm Top Rules cho từng cụm)
print("\n=== ĐẶC ĐIỂM HÀNH VI (Gợi ý viết báo cáo) ===")
for cluster_id in sorted(df_res['cluster'].unique()):
    n_cust = rfm_stats.loc[cluster_id, 'Số Lượng Khách']
    avg_money = rfm_stats.loc[cluster_id, 'Monetary']
    
    print(f"\n--- CỤM {cluster_id} ({n_cust} khách) ---")
    if avg_money > 5000:
        print(">> LOẠI KHÁCH: VIP / Mua Sỉ (Big Spenders)")
        print(">> CHIẾN LƯỢC: Chăm sóc đặc biệt, chiết khấu theo tier.")
    elif n_cust > 1000:
        print(">> LOẠI KHÁCH: Phổ thông (Casual)")
        print(">> CHIẾN LƯỢC: Gửi mã giảm giá, upsell sản phẩm nhỏ.")
    else:
        print(">> LOẠI KHÁCH: Ngách (Niche)")

In [ ]:
# --- CODE SỬA LỖI Ô 14: PHÂN CỤM LUẬT (RULE CLUSTERING) ---
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

print("\n=== THỰC HIỆN YÊU CẦU NÂNG CAO 2: PHÂN CỤM LUẬT ===")

# 1. Load lại file luật (Sửa lỗi thiếu biến)
try:
    rules_path = "data/processed/rules_apriori_filtered.csv"
    rules = pd.read_csv(rules_path)
    print(f"Đã load {len(rules)} luật từ file.")

    # 2. Chuẩn bị dữ liệu (Chỉ dùng Support, Confidence, Lift)
    rule_features = rules[['support', 'confidence', 'lift']].copy()
    
    # 3. Chuẩn hóa & Phân cụm
    scaler_rules = StandardScaler()
    X_rules = scaler_rules.fit_transform(rule_features)
    
    # Chia thành 3 nhóm luật điển hình
    kmeans_rules = KMeans(n_clusters=3, random_state=42, n_init='auto')
    rules['rule_cluster'] = kmeans_rules.fit_predict(X_rules)
    
    # 4. Thống kê kết quả
    print("\n>>> KẾT QUẢ PHÂN NHÓM CÁC LUẬT (Dùng để chọn chiến lược Promotion):")
    rule_summary = rules.groupby('rule_cluster').agg({
        'rule_str': 'count',
        'support': 'mean',
        'confidence': 'mean',
        'lift': 'mean'
    }).rename(columns={'rule_str': 'Số lượng Luật', 'lift': 'Lift TB'}).sort_values('Lift TB', ascending=False).round(3)
    
    display(rule_summary)
    
    print("\n=> NHẬN XÉT: Nhóm luật có Lift cao nhất là nhóm 'Luật Vàng' cần ưu tiên hiển thị trên web.")

except Exception as e:
    print(f"Lỗi: {e}")